In [ ]:

# @title SETUP
!pip install git+https://github.com/huggingface/diffusers.git transformers xformers accelerate omegaconf torchsde
!apt -y install -qq aria2

In [ ]:
import os
os.makedirs(f'models',exist_ok=True)

# @title DOWNLOAD MODEL

Model_url = 'https://civitai.com/api/download/models/119057'# @param {type:"string"}
Model_name = 'meina_MIX'# @param {type:"string"}
if not Model_name.endswith('.safetensors'):
    Model_name=Model_name+'.safetensors'

!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {Model_url} -d models -o {Model_name}
print('\nAvailable models :')
for MODEL in os.listdir('models'):
    print(MODEL.replace('.safetensors',''))

In [ ]:
# @title LOAD MODEL

from diffusers import StableDiffusionPipeline, DDIMParallelScheduler, DDIMScheduler, DDPMParallelScheduler, DDPMScheduler, DEISMultistepScheduler, DPMSolverMultistepScheduler, DPMSolverSDEScheduler, DPMSolverSinglestepScheduler, EulerAncestralDiscreteScheduler, EulerDiscreteScheduler, HeunDiscreteScheduler, KDPM2AncestralDiscreteScheduler, KDPM2DiscreteScheduler, LMSDiscreteScheduler, PNDMScheduler, UniPCMultistepScheduler
from diffusers.utils import make_image_grid
import torch,random,os
from IPython.display import display

os.makedirs(f'outputs',exist_ok=True)
MODEL_NAME  = "meina_MIX"# @param {type:"string"}
LOAD_TYPE = "FP16 (FAST BUT AVERAGE QUALITY)"# @param ["FP32 (SLOW BUT GOOD QUALITY)","FP16 (FAST BUT AVERAGE QUALITY)"]
if not MODEL_NAME.endswith('.safetensors'):
    MODEL_NAME=MODEL_NAME+'.safetensors'
print(f'\n\nCurrently selected : {MODEL_NAME}')
model_path = f'models/{MODEL_NAME}'
if LOAD_TYPE == "FP32 (SLOW BUT GOOD QUALITY)":
    pipe = StableDiffusionPipeline.from_single_file(model_path,load_safety_checker = False,scheduler_type='dpm')
else:
    pipe = StableDiffusionPipeline.from_single_file(model_path, revision="fp16",load_safety_checker = False, torch_dtype=torch.float16,variant="fp16",scheduler_type='dpm')
pipe.to("cuda")
pipe.enable_xformers_memory_efficient_attention()

In [ ]:
# @title GENERATE IMAGE

PROMPT= ''# @param {type:"string"}
NEGATIVE_PROMPT = ''# @param {type:"string"}
WIDTH  = 512# @param {type:"integer"}
HEIGHT = 512 # @param {type:"integer"}
SAMPLING_STEPS  = 30# @param {type:"integer"}
CFG_scale  = 5# @param {type:"number"}
SEED  = -1# @param {type:"number"}
CLIP_SKIP  = 1# @param {type:"number"}
SCHEDULER  = "DPMSolverMultistepScheduler"# @param ["DDIMParallelScheduler","DDIMScheduler","DDPMParallelScheduler","DDPMScheduler","DEISMultistepScheduler","DPMSolverMultistepScheduler","DPMSolverSDEScheduler","DPMSolverSinglestepScheduler","EulerAncestralDiscreteScheduler","EulerDiscreteScheduler","HeunDiscreteScheduler","KDPM2AncestralDiscreteScheduler","KDPM2DiscreteScheduler","LMSDiscreteScheduler","PNDMScheduler","UniPCMultistepScheduler"]
USE_KARRAS = True # @param {type:"boolean"}
NUMBER_OF_IMAGES  = 4# @param {type:"number"}
CLIP_SKIP = None if CLIP_SKIP == 1 else CLIP_SKIP - 1

if SEED == -1:
    SEED = random.randint(1,10000000000)
generator = torch.Generator(device="cuda").manual_seed(SEED)
sc = f'''pipe.scheduler = {SCHEDULER}.from_config(pipe.scheduler.config, use_karras_sigmas={USE_KARRAS})'''
exec(sc)
final_images=[]
if NUMBER_OF_IMAGES != 1:
    for i in range(NUMBER_OF_IMAGES):
        SEED = random.randint(1,10000000000)
        print(f'SEED : {SEED}')
        generator = torch.Generator(device="cuda").manual_seed(SEED)
        image = pipe(PROMPT,negative_prompt = NEGATIVE_PROMPT,clip_skip=CLIP_SKIP,generator=generator,width = WIDTH , height = HEIGHT,num_inference_steps=SAMPLING_STEPS,guidance_scale = CFG_scale).images[0]
        final_images.append(image)
        image.save(f'outputs/{PROMPT[:10]}_{SEED}.png')
    f_images = make_image_grid(final_images, 1, len(final_images))
    display(f_images)
else:
    print(f'SEED : {SEED}')
    image = pipe(PROMPT,negative_prompt = NEGATIVE_PROMPT,clip_skip=CLIP_SKIP,generator=generator,width = WIDTH , height = HEIGHT,num_inference_steps=SAMPLING_STEPS,guidance_scale = CFG_scale).images[0]
    image.save(f'outputs/{PROMPT[:11]}_{SEED}_{CLIP_SKIP}_{SCHEDULER}.png')
    display(image)


In [ ]:
# @title DOWNLOAD THE ZIP WITH ALL THE IMAGES
import zipfile
import os
from google.colab import files

def zip_folder(folder_path, zip_filename):
    with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, _, files in os.walk(folder_path):
            for file in files:
                file_path = os.path.join(root, file)
                arcname = os.path.relpath(file_path, folder_path)
                zipf.write(file_path, arcname)

folder_to_zip = "outputs"
zip_filename = "output.zip"
zip_folder(folder_to_zip, zip_filename)
print(f'Succesfully saved all the images in {zip_filename}\nDownloading the zip.....')
files.download(zip_filename)